<center><img src='https://www.datahack.es/wp-content/uploads/2020/02/logo-home.png' width="500"></center>

# Ejercicio de algoritmos basados en contenido 2 PUNTOS
En este ejercicio os propongo generar un recomendadador basado en contenido, con los datos de la popular web www.xataka.com.

Para ello he descargado las etiquetas de los artículos de los últimos 13 meses. 

<center><img src='https://i.blogs.es/11ca44/xataka/450_1000.jpg' width=600 ></center>

## Información

La información está almacenada de manera que cada fichero representa un mes, y contiene la siguiente información guardada en formato json :

- **artículo:** es la parte final de la url del artículo, que si le pegamos por delante el prefijo "xataka.com" nos llevaría directamene a la web del artículo.

- **etiquetas:** son las etiquetas asociadas al artículo, las hay de dos tipos , tags y categoría. Se pueden usar indistintamente o si creeís que puede aportar algo de información tratarlas independientemente.

- **comentarios:** es una info adicional que se podría usar o no para mejorar la recomendación. 

- **nombre del ficheros:** el nombre del fichero nos indica el año y el mes del artículo. Esta información también la podríamos usar para mejorar la recomendación, no es obligatorio.

## Pasos a seguir

1. Descarga la información de xataka en una carpeta que se llame `datos_ejercicios/xataka`.

2. Lee los ficheros json utilizando la función spark.read.json y unifícalos en un único dataframe.

3. Crea una columnas de nombre idx, que se corresponda con row number del dataset.

4. Genera un algortimo basado en contenido (no olvides poner comentarios en el código). Explica el motivo de cada paso. Puedes hacer uso de todas la información. Se valora la creatividad.

5. Crea una función que dado un idx nos devuelva el nombre del artículo junto con el nombre del top 5 de artículos similares. Se valora la creatividad.

6. Descarga el ipynb, conviertelo a html y esos serán tus documentos de evaluación.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##0. Instalaciones y librerías

Instalación de Spark, Findspark y Pyspark.

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 204.2MB 67kB/s 
     |████████████████████████████████| 204kB 43.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=757f02336bf39f3fe7afe67a95d0ec737133ca8616d283de8d8fdd96db98a75d
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


Creación de la Sesión Spark "ejercicio_recomendadores_tipo_ranking_rehecho".

In [4]:
# Set up required environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"
from pyspark import SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ejercicio_recomendador_basado_en_contenido").master("local[*]").getOrCreate()

In [5]:
#Carga de datos de la web, no lo usamos en esta práctica (ejemplo):
#!pip install wget 
#import wget

#Tratamiento de datos:
import pandas as pd
import numpy as np
import pyspark.sql.functions as F
import pyspark.sql.types as T
import glob
from pyspark.ml.feature import CountVectorizer
import re

#Dibujo de tablas y gráficos:
pd.set_option('display.max_rows', 50) #max 50 rows out


1. Descarga la información de IMDb en una carpeta que se llame datos_ejercicios/imdb.

In [6]:
#No lo hacemos, usaremos la data almacenada en la carpeta de la práctica, pero sería

#url= "url"
#wget.download(url, "/content/drive/My Drive/Colab Notebooks/Práctica Spark/datos_ejercicios/xataka")

# <font color="Grey">2. Lee los ficheros json utilizando la función spark.read.json y unifícalos en un único dataframe.

<font color="Grey">Cargamos todos los paths de los 13 archivos json en la variable "archivos", mediante un bucle vamos a leerlos con Spark.read.json, pasarlos a Pandas e incluirlos en una lista que llamaremos variable "lista_data". Después los concatenaremos.  
Realizaremos un estudio breve de los datos, presencia de nulos y eliminado de duplicados.

In [7]:
archivos= glob.glob('/content/drive/My Drive/Colab Notebooks/Práctica Spark/datos_ejercicios/xataka/*.json')

In [8]:
archivos

['/content/drive/My Drive/Colab Notebooks/Práctica Spark/datos_ejercicios/xataka/2019_2.json',
 '/content/drive/My Drive/Colab Notebooks/Práctica Spark/datos_ejercicios/xataka/2019_9.json',
 '/content/drive/My Drive/Colab Notebooks/Práctica Spark/datos_ejercicios/xataka/2020_1.json',
 '/content/drive/My Drive/Colab Notebooks/Práctica Spark/datos_ejercicios/xataka/2019_6.json',
 '/content/drive/My Drive/Colab Notebooks/Práctica Spark/datos_ejercicios/xataka/2019_10.json',
 '/content/drive/My Drive/Colab Notebooks/Práctica Spark/datos_ejercicios/xataka/2019_8.json',
 '/content/drive/My Drive/Colab Notebooks/Práctica Spark/datos_ejercicios/xataka/2019_4.json',
 '/content/drive/My Drive/Colab Notebooks/Práctica Spark/datos_ejercicios/xataka/2019_5.json',
 '/content/drive/My Drive/Colab Notebooks/Práctica Spark/datos_ejercicios/xataka/2019_3.json',
 '/content/drive/My Drive/Colab Notebooks/Práctica Spark/datos_ejercicios/xataka/2019_7.json',
 '/content/drive/My Drive/Colab Noteboo

In [9]:
lista_data=[]
for i in archivos:
  m=re.search(r'(?<=/)\d+_\d+',i)
  fecha=m.group(0)
  dfs=spark.read.json(i).toPandas()
  dfs["fecha"]=fecha
  lista_data.append(dfs)

In [10]:
df=pd.concat(lista_data,ignore_index=True)

In [11]:
df

,articulo,comentarios,etiquetas,fecha
0,/automovil/tesla-finalmente-lanza-version-35-0...,101,"[/categoria/automovil, /tag/tesla, /tag/coche-...",2019_2
1,/servicios/youtube-tratara-evitar-que-videos-m...,13,"[/categoria/servicios, /tag/google, /tag/ninos...",2019_2
2,/moviles/vitrina-world-congress-2019-ano-que-n...,10,"[/categoria/moviles, /tag/pantallas-flexibles,...",2019_2
3,/inteligencia-artificial/cara-cruz-ia-pib-mund...,41,"[/categoria/inteligencia-artificial, /tag/espa...",2019_2
4,/basics/como-remapear-boton-bixby-tu-samsung-g...,2,"[/categoria/basics, /tag/samsung, /tag/bixby, ...",2019_2
...,...,...,...,...
6602,/domotica-1/estas-luces-navidad-permiten-usar-...,9,"[/categoria/domotica-1, /categoria/internet-of...",2020_12
6603,/legislacion-y-derechos/congreso-eeuu-amenaza-...,83,"[/categoria/legislacion-y-derechos, /categoria...",2020_12
6604,/vehiculos/que-busca-convertirse-primer-avion-...,15,"[/categoria/vehiculos, /tag/canada, /tag/avion...",2020_12
6605,/medicina-y-salud/celulas-se-le-volvieron-loca...,10,"[/categoria/medicina-y-salud, /tag/medicina, /...",2020_12


In [12]:
#Voy a crear una columna con las urls
url= ['xataka.com'+i for i in df.articulo]
df['url']=url
df

,articulo,comentarios,etiquetas,fecha,url
0,/automovil/tesla-finalmente-lanza-version-35-0...,101,"[/categoria/automovil, /tag/tesla, /tag/coche-...",2019_2,xataka.com/automovil/tesla-finalmente-lanza-ve...
1,/servicios/youtube-tratara-evitar-que-videos-m...,13,"[/categoria/servicios, /tag/google, /tag/ninos...",2019_2,xataka.com/servicios/youtube-tratara-evitar-qu...
2,/moviles/vitrina-world-congress-2019-ano-que-n...,10,"[/categoria/moviles, /tag/pantallas-flexibles,...",2019_2,xataka.com/moviles/vitrina-world-congress-2019...
3,/inteligencia-artificial/cara-cruz-ia-pib-mund...,41,"[/categoria/inteligencia-artificial, /tag/espa...",2019_2,xataka.com/inteligencia-artificial/cara-cruz-i...
4,/basics/como-remapear-boton-bixby-tu-samsung-g...,2,"[/categoria/basics, /tag/samsung, /tag/bixby, ...",2019_2,xataka.com/basics/como-remapear-boton-bixby-tu...
...,...,...,...,...,...
6602,/domotica-1/estas-luces-navidad-permiten-usar-...,9,"[/categoria/domotica-1, /categoria/internet-of...",2020_12,xataka.com/domotica-1/estas-luces-navidad-perm...
6603,/legislacion-y-derechos/congreso-eeuu-amenaza-...,83,"[/categoria/legislacion-y-derechos, /categoria...",2020_12,xataka.com/legislacion-y-derechos/congreso-eeu...
6604,/vehiculos/que-busca-convertirse-primer-avion-...,15,"[/categoria/vehiculos, /tag/canada, /tag/avion...",2020_12,xataka.com/vehiculos/que-busca-convertirse-pri...
6605,/medicina-y-salud/celulas-se-le-volvieron-loca...,10,"[/categoria/medicina-y-salud, /tag/medicina, /...",2020_12,xataka.com/medicina-y-salud/celulas-se-le-volv...


In [13]:
df_spark= spark.createDataFrame(df)

In [14]:
df_spark.show()

+--------------------+-----------+--------------------+------+--------------------+
|            articulo|comentarios|           etiquetas| fecha|                 url|
+--------------------+-----------+--------------------+------+--------------------+
|/automovil/tesla-...|        101|[/categoria/autom...|2019_2|xataka.com/automo...|
|/servicios/youtub...|         13|[/categoria/servi...|2019_2|xataka.com/servic...|
|/moviles/vitrina-...|         10|[/categoria/movil...|2019_2|xataka.com/movile...|
|/inteligencia-art...|         41|[/categoria/intel...|2019_2|xataka.com/inteli...|
|/basics/como-rema...|          2|[/categoria/basic...|2019_2|xataka.com/basics...|
|/televisores/bloa...|         56|[/categoria/telev...|2019_2|xataka.com/televi...|
|/otros/cable-subm...|         19|[/categoria/otros...|2019_2|xataka.com/otros/...|
|/aplicaciones/chr...|         13|[/categoria/aplic...|2019_2|xataka.com/aplica...|
|/seleccion/como-i...|         11|[/categoria/selec...|2019_2|xataka.com/sel

In [15]:
#Vemos si hay nulos
df_spark.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df_spark.columns]).show()

+--------+-----------+---------+-----+---+
|articulo|comentarios|etiquetas|fecha|url|
+--------+-----------+---------+-----+---+
|       0|          0|        0|    0|  0|
+--------+-----------+---------+-----+---+



In [16]:
#Vamos a ver si hay artículos duplicados
if df_spark.count() == df_spark.select('articulo').distinct().count():
  print ('No hay duplicados')
else:
  print('Existen duplicados')
  print('Nº de duplicados:',df_spark.count()-df_spark.select('articulo').distinct().count())

Existen duplicados
Nº de duplicados: 1


In [17]:
#Buscamos cuál está duplicado
df_spark.groupBy(df_spark.articulo).count().filter(F.col('count')!=1).show(truncate=False)

+----------------------------------------------------------------------------------------------------------+-----+
|articulo                                                                                                  |count|
+----------------------------------------------------------------------------------------------------------+-----+
|/seleccion/guia-compra-accesorios-para-nintendo-switch-lite-27-fundas-carcasas-mandos-soportes-tarjetas-sd|2    |
+----------------------------------------------------------------------------------------------------------+-----+



In [18]:
#Vamos a ver este duplicado en el total del dataset para ver si esta duplicada toda la row:
duplicado= df_spark.groupBy(df_spark.articulo).count().filter(F.col('count')!=1).collect()[0][0] #Con esto sacamos el valor del articulo para hacer un filter
df_spark.filter(F.col('articulo')==duplicado).show(truncate=False)

+----------------------------------------------------------------------------------------------------------+-----------+---------------------------------------------------------------------------------------+-------+--------------------------------------------------------------------------------------------------------------------+
|articulo                                                                                                  |comentarios|etiquetas                                                                              |fecha  |url                                                                                                                 |
+----------------------------------------------------------------------------------------------------------+-----------+---------------------------------------------------------------------------------------+-------+--------------------------------------------------------------------------------------------------------------------

In [19]:
#Tienen años diferentes, pero como queremos recomendar artículos, podría darse el caso de que salieran los dos en una misma recomendación, por lo que vamos a cargarnos el más antiguo,
#para ello, reordenamos el dataset de más moderno a más antiguo y luego eliminamos(ya que elimina el segundo duplicado)
df_spark=df_spark.orderBy(F.desc('fecha')).drop_duplicates(subset=['articulo'])

In [20]:
df_spark.filter(F.col('articulo')==duplicado).show(truncate=False)

+----------------------------------------------------------------------------------------------------------+-----------+---------------------------------------------------------------------------------------+-------+--------------------------------------------------------------------------------------------------------------------+
|articulo                                                                                                  |comentarios|etiquetas                                                                              |fecha  |url                                                                                                                 |
+----------------------------------------------------------------------------------------------------------+-----------+---------------------------------------------------------------------------------------+-------+--------------------------------------------------------------------------------------------------------------------

In [21]:
df_spark.printSchema()

root
 |-- articulo: string (nullable = true)
 |-- comentarios: long (nullable = true)
 |-- etiquetas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- fecha: string (nullable = true)
 |-- url: string (nullable = true)



# <font color="Grey">3. Crea una columnas de nombre idx, que se corresponda con row number del dataset.

In [22]:
df_spark = (df_spark
            .withColumn("idx", F.monotonically_increasing_id())
            .select("idx","articulo","comentarios","etiquetas","fecha","url")
)

In [23]:
df_spark.show()

+---+--------------------+-----------+--------------------+-------+--------------------+
|idx|            articulo|comentarios|           etiquetas|  fecha|                 url|
+---+--------------------+-----------+--------------------+-------+--------------------+
|  0|/audio/midi-2-0-s...|         18|[/categoria/audio...| 2020_1|xataka.com/audio/...|
|  1|/basics/como-borr...|         14|[/categoria/basic...|2019_10|xataka.com/basics...|
|  2|/basics/redaccion...|          1|[/categoria/basic...| 2019_3|xataka.com/basics...|
|  3|/domotica-1/dos-a...|         30|[/categoria/domot...|2020_11|xataka.com/domoti...|
|  4|/ecologia-y-natur...|         37|[/categoria/ecolo...|2020_12|xataka.com/ecolog...|
|  5|/empresas-y-econo...|         34|[/categoria/empre...| 2019_7|xataka.com/empres...|
|  6|/espacio/heavy-ro...|         25|[/categoria/espac...| 2019_4|xataka.com/espaci...|
|  7|/espacio/stratola...|         16|[/categoria/espac...| 2019_4|xataka.com/espaci...|
|  8|/inteligencia-ar

# <font color="Grey">4. Genera un algortimo basado en contenido (no olvides poner comentarios en el código). Explica el motivo de cada paso. Puedes hacer uso de todas la información. Se valora la creatividad.

<font color="Grey">Vamos a usar la función CountVectorizer, ya que usaremos las etiquetas para basar nuestras recomendaciones. Esta función se basa en el número de veces que se repite un contenido, en nuestro caso una categoría o un tag, sin dar más importancia a unos que a otros. Un contenido que tiene muchos tags en común con otro quiere decir que son contenidos parecidos.

In [24]:
from pyspark.ml.feature import CountVectorizer

# Entrenamos el CountVectorizerModel desde el corpus.
cv=CountVectorizer( inputCol='etiquetas', outputCol="features")
model=cv.fit(df_spark)
df_cv=model.transform(df_spark).cache()
df_cv.show(truncate=False)

+---+----------------------------------------------------------------------------------------------------------------------------------------+-----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+--------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------+
|idx|articulo                                                                                                                                |comentarios|etiquetas                                                                                                                                                                                |fecha  |url                                                                    

#<font color="grey">5. Crea una función que dado un idx nos devuelva el nombre del artículo junto con el nombre del top 5 de artículos similares. Se valora la creatividad.

<font color="Grey">En primer lugar, vamos a crear nuestra función dot_udf que calcula la similitud para cada par de contenidos.

In [25]:
dot_udf = F.udf(lambda x,y: float(x.dot(y)), T.DoubleType())

<font color="Grey">Creamos una función con la que, dado un idx de un artículo, nos ofrece los 5 artículos similares.  
Realizamos un crossJoin de nuetro df_cv consigo misma para así tener todos los artículos enfrentados con todos en una misma fila, generando un dos nuevos índices, que serán el índice del artículo_1 (del que buscamos contenido similar) y un idx_2 que será el del artículo similar, pero su url, para que el usuario pueda clicar y redirigirse al contenido recomendado. Además, incluiremos la fecha, para aportar información al usuario.  
Generaremos la columna dot, que muestra el grado de relación entre los artículos (resultado de la función dot_udf) y lo ordenaremos descendentemente para mostrar la mayor relación al principio de nuestra tabla.  
Por último, mostraremos los 5 primeros resultados como nos pide el ejercicio.


In [26]:
def Top5_artículos_similares(idx_articulo):
  """
  Función que dado un artículo muestra el Top5
  de artículos similares
  """
  (df_cv.alias("i").crossJoin(df_cv.alias("j"))
        .select(
        F.col("i.idx").alias("idx_1"), 
        F.col("j.idx").alias("idx_2"),
        F.col("i.articulo").alias("articulo_1"),  
        F.col("j.url").alias("recomendaciones"),  
        F.col("i.fecha"),
        #.orderBy("idx_1",F.desc("fecha")),
        dot_udf("i.features", "j.features").alias("afinidad"))
        .orderBy("idx_1",F.desc("afinidad"))  
        .filter("idx_1!=idx_2")
        .filter("idx_1 = %s" %idx_articulo)
  ).show(5,truncate=False)

In [27]:
Top5_artículos_similares(1)

+-----+------------+------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------+-------+--------+
|idx_1|idx_2       |articulo_1                                                        |recomendaciones                                                                                                               |fecha  |afinidad|
+-----+------------+------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------+-------+--------+
|1    |137438953482|/basics/como-borrar-informacion-personal-tuya-que-aparece-internet|xataka.com/basics/como-solicitar-que-no-se-te-envie-propaganda-electoral-correo-postal-internet                               |2019_10|3.0     |
|1    |455266533397|/basics/como-borrar-informacion-personal-tuya-que-ap